# Mobility-Station-Finder

## Imports

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
from collections import namedtuple

## Read and process static data

### Paths

In [ ]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_jrta = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\140_JRTA_(OEV).mtx"
path_pt_ntr = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\144_NTR_(OEV).mtx"

### Read NPVM-zones

In [ ]:
df_npvm_zones = gpd.read_file(path_npvm_zones).to_crs(4326)

In [ ]:
df_npvm_zones.head()

In [ ]:
len(df_npvm_zones)

In [ ]:
df_npvm_zones[df_npvm_zones.ID == 5301003]

### Read an process Mobility-stations

In [ ]:
df_mobility = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].groupby("Stationsnummer").first().dropna()

In [ ]:
df_mobility["lon"] = df_mobility["Standort"].apply(lambda x: x.split(",")[1])
df_mobility["lat"] = df_mobility["Standort"].apply(lambda x: x.split(",")[0])

In [ ]:
df_mobility

In [ ]:
gdf_mobility = gpd.GeoDataFrame(df_mobility, geometry=gpd.points_from_xy(df_mobility.lon, df_mobility.lat), crs=4326)

In [ ]:
gdf_mobility.head()

### Assign NPVM-zone to Mobility-Stations

In [ ]:
mobility_zone = gpd.sjoin(gdf_mobility, df_npvm_zones)

In [ ]:
mobility_zone

In [ ]:
mobility_zone.to_csv(r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Analyse_Mobility_Stationen.csv", sep=";")

### Read PT-skims

In [ ]:
skim_jrta = ReadPTVMatrix(path_pt_jrta)

In [ ]:
skim_ntr = ReadPTVMatrix(path_pt_ntr)

In [ ]:
skim_jrta.values

In [ ]:
skim_jrta.sel(origins=378601001).sel(destinations=35101026).matrix.item()

In [ ]:
skim_ntr.sel(origins=378601001).sel(destinations=35101026).matrix.item()

In [ ]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [ ]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [ ]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [ ]:
get_jrta(378601001, 35101026)

In [ ]:
get_ntr(378601001, 35101026)

### Read relevant skims on relevant od-pairs

In [ ]:
from_npvm_zones = df_npvm_zones["ID"].to_list()
to_npvm_zones = mobility_zone["ID"].to_list()

In [ ]:
nb_from_zones = len(from_npvm_zones)
nb_to_zones = len(to_npvm_zones)
nb_od_pairs = nb_from_zones * nb_to_zones
print(nb_from_zones, nb_to_zones, nb_od_pairs)

## Execute query

### Define origin and destination and assign NPVM-Zone

In [ ]:
orig_point = Point(7.423570, 46.936620)
dest_point = Point(7.695260, 46.828540)

In [ ]:
df_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
df_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [ ]:
orig_zone = gpd.sjoin(df_orig, df_npvm_zones)
dest_zone = gpd.sjoin(df_dest, df_npvm_zones)

In [ ]:
dest_zone

### Compute potential Mobility-stations in a circle around origin and destination

In [ ]:
search_area_orig = gpd.GeoDataFrame(geometry=df_orig.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
search_area_dest = gpd.GeoDataFrame(geometry=df_dest.to_crs(2026).buffer(50*1000).to_crs(4326), crs="EPSG:4326")

In [ ]:
orig_zone_mobility_stations = gpd.sjoin(gdf_mobility, search_area_orig)
dest_zone_mobility_stations = gpd.sjoin(gdf_mobility, search_area_dest)

In [ ]:
orig_zone_mobility_stations

In [ ]:
len(orig_zone_mobility_stations)

In [ ]:
len(dest_zone_mobility_stations)

TODO:
- Join potential Mobility-stations
- Open Routing Machine query from every potential Mobility-station to destination
- Skims for NPVM-zone of origin to every NPVM-zone assigned to a potential Mobility-station
- Calculate generalized costs

### Visualize situation on map

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import geopandas
import json


m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)

geo_data = GeoData(geo_dataframe = df_npvm_zones,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Countries')

m.add_layer(geo_data)
m.add_control(LayersControl())

In [ ]:
m.layout.width = '100%'
m.layout.height = '1000px'
m
# m.save("npvm-zonen.html")